### Code from Bi-directional LSTM tutorial 
https://machinelearningmastery.com/develop-bidirectional-lstm-sequence-classification-python-keras/


In [22]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential

# Traditional LSTM
# Given a sequence of random values between 0 and 1,
# assign an output of 0,
# sum up the sequence,
# when a threshold is reached, assign an output of 1

def get_sequence(n_timesteps):
    X = array([random() for _ in range(n_timesteps)])
    limit = n_timesteps/4.0
    y = array([0 if x < limit else 1 for x in cumsum(X)])    
    
    X = X.reshape(1, n_timesteps, 1)
    y = y.reshape(1, n_timesteps, 1)

    return X, y


In [21]:
X, y = get_sequence(10)
print(X)
print(y)

[[[ 0.28238311]
  [ 0.26553763]
  [ 0.04702921]
  [ 0.71302156]
  [ 0.41869366]
  [ 0.58109325]
  [ 0.76037344]
  [ 0.72475499]
  [ 0.05964365]
  [ 0.77431409]]]
[[[0]
  [0]
  [0]
  [0]
  [0]
  [0]
  [1]
  [1]
  [1]
  [1]]]


In [ ]:
model = Sequential()
model.add(LSTM(20, input_shape(10,1), return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='acc')

for epoch in range(1000):
    X, y = get_sequence(n_timesteps)
    model.fit(X, y, epochs=1, batch_size=1, verbose=2)


X, y = get_sequence(n_timesteps)
yhat = model.predict_classes(X, verbose=0)

for i in range(n_timesteps):
    print('Expected:', y[0, i], 'Predicted', yhat[0, 1])